# CMPE 351 Assignment 2

We'll first import the necessary python packages to run the code in the
notebook.


In [2]:
# Importing packages.
from os.path import dirname, join, realpath

import pandas as pd
import torch
import torch.nn as nn
import torchvision
from skimage import io, transform
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import transforms

In [ ]:
def is_interactive():
    import __main__ as main

    return not hasattr(main, "__file__")


if is_interactive():
    script_dir = dirname(realpath("__file__"))
else:
    script_dir = dirname(realpath(__file__))


## Part 1: Image Classification using CNN (50 points)

In [ ]:
# Hyper parameters
epochs_count = 2
classes_count = 10
batch_size = 32
learning_rate = 0.001

In [ ]:
class FashionProductDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.fashion_products_data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.fashion_products_data_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = join(
            self.root_dir, self.fashion_products_data_frame.iloc[idx, 0]
        )
        image = io.imread(img_name)
        landmarks = self.fashion_products_data_frame.iloc[idx, 1:]
        landmarks = np.array([landmarks])
        landmarks = landmarks.astype("float").reshape(-1, 2)
        sample = {"image": image, "landmarks": landmarks}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [ ]:
training_data_frame = pd.read_csv(join(script_dir, "data/train.csv"))
test_data_frame = pd.read_csv(join(script_dir, "data/test.csv"))
training_dataset = torchvision.datasets.MNIST(
    root="./data", train=True, transform=transforms.ToTensor(), download=True
)